In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import torch
from datetime import datetime, timedelta
from typing import List, Tuple, Union, Dict
import time

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from darts import TimeSeries
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.dataprocessing.transformers.scaler import Scaler
from darts.models import TiDEModel, NaiveMovingAverage, TFTModel
from darts.metrics import mae, mse, smape
from darts.utils.losses import MAELoss, MapeLoss, SmapeLoss
import darts

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import torch
import torch.nn.functional as F
import torchmetrics
from torch import nn

def set_global_seed(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # CuDNN 연산을 deterministic하게 만들어 주지만, 약간 느려질 수 있음
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 값 고정
set_global_seed()

c:\Users\tjqtj\miniconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_darts_time_series_group(
    dataset: pd.DataFrame,
    target: Union[List[str],str],
    time_col: str,
    group_cols: Union[List[str],str],
    static_cols: Union[List[str],str]=None,
    past_cols: Union[List[str],str]=None,
    future_cols: Union[List[str],str]=None,
    freq: str=None,
    encode_static_cov: bool=True,
)-> Tuple[List[TimeSeries], List[TimeSeries], List[TimeSeries], List[TimeSeries]]:

    series_raw = TimeSeries.from_group_dataframe(
    dataset,
    time_col    =   time_col,
    group_cols  =   group_cols,  # individual time series are extracted by grouping `df` by `group_cols`
    static_cols =   static_cols,  # also extract these additional columns as static covariates (without grouping)
    value_cols  =   target,  # optionally, specify the time varying columns
    n_jobs      =   -1,
    verbose     =   False,
    freq        =   freq,
    )

    if encode_static_cov:
        static_cov_transformer = StaticCovariatesTransformer()
        series_encoded = static_cov_transformer.fit_transform(series_raw)
    else: series_encoded = []

    if past_cols:
        past_cov = TimeSeries.from_group_dataframe(
            dataset,
            time_col    =   time_col,
            group_cols  =   group_cols,
            value_cols  =   past_cols,
            n_jobs      =   -1,
            verbose     =   False,
            freq        =   freq,
            )
    else: past_cov = []

    if future_cols:
        future_cov = TimeSeries.from_group_dataframe(
            dataset,
            time_col    =   time_col,
            group_cols  =   group_cols,
            value_cols  =   future_cols,
            n_jobs      =   -1,
            verbose     =   False,
            freq        =   freq,
            )
    else: future_cov = []

    return series_raw, series_encoded, past_cov, future_cov

def split_grouped_darts_time_series(
    series: List[TimeSeries],
    split_date: Union[str, pd.Timestamp],
    min_date: Union[str, pd.Timestamp]=None,
    max_date: Union[str, pd.Timestamp]=None,
) -> Tuple[List[TimeSeries], List[TimeSeries]]:

    if min_date:
       raw_series = series.copy()
       series = []
       for s in raw_series:
        try: series.append(s.split_before(pd.Timestamp(min_date)-timedelta(1))[1])
        except: series.append(s)

    if max_date:
       raw_series = series.copy()
       series = []
       for s in raw_series:
        try: series.append(s.split_before(pd.Timestamp(max_date))[0])
        except: series.append(s)

    split_0 = [s.split_before(pd.Timestamp(split_date))[0] for s in series]
    split_1 = [s.split_before(pd.Timestamp(split_date))[1] for s in series]
    return split_0, split_1

def eval_forecasts(
    pred_series: Union[List[TimeSeries], TimeSeries],
    test_series: Union[List[TimeSeries], TimeSeries],
    error_metric: darts.metrics,
    plot: bool=False
) -> List[float]:

    errors = error_metric(test_series, pred_series)
    print(errors)
    if plot:
        plt.figure()
        plt.hist(errors, bins=50)
        plt.ylabel("Count")
        plt.xlabel("Error")
        plt.title(f"Mean error: {np.mean(errors):.3f}")
        plt.show()
        plt.close()
    return errors

def fit_mixed_covariates_model(
    model_cls,
    common_model_args: dict,
    specific_model_args: dict,
    model_name: str,
    past_cov: Union[List[TimeSeries], TimeSeries],
    future_cov: Union[List[TimeSeries], TimeSeries],
    train_series: Union[List[TimeSeries], TimeSeries],
    val_series: Union[List[TimeSeries], TimeSeries]=None,
    max_samples_per_ts: int=None,
    save:bool=False,
    path:str="",
):

    # Declarare model
    model = model_cls(model_name=model_name,
                    **common_model_args,
                    **specific_model_args)

    # Train model
    model.fit(
                    # TRAIN ARGS ===================================
                    series                = train_series,
                    past_covariates       = past_cov,
                    future_covariates     = future_cov,
                    max_samples_per_ts    = max_samples_per_ts,
                    # VAL ARGS ======================================
                    val_series            = val_series,
                    val_past_covariates   = past_cov,
                    val_future_covariates = future_cov,
                )

    if save: model.save(path)

def backtesting(model, series, past_cov, future_cov, start_date, horizon, stride):
  historical_backtest = model.historical_forecasts(
    series, past_cov, future_cov,
    start=start_date,
    forecast_horizon=horizon,
    stride=stride,  # Predict every N months
    retrain=False,  # Keep the model fixed (no retraining)
    overlap_end=False,
    last_points_only=False
  )
  maes = model.backtest(series, historical_forecasts=historical_backtest, metric=mae)

  return np.mean(maes)

def process_predictions(
    preds: List[TimeSeries],
    series_raw: List[TimeSeries],
    group_cols: List[str]
) -> pd.DataFrame:

    list_df = [serie.pd_dataframe() for serie in preds]
    for i in range(len(list_df)):
      list_df[i]['Date'] = preds[i].time_index
      for j in range(len(group_cols)):
        list_df[i][group_cols[j]] = series_raw[i].static_covariates[group_cols[j]].values[0]
    processed_preds =  pd.concat(list_df, ignore_index=True)
    return processed_preds

def price_weighted_mae(predictions, targets, prices):
    """
    Compute the price-weighted Mean Absolute Error (MAE).

    :param predictions: A list or 1D NumPy array of predicted values.
    :param targets: A list or 1D NumPy array of actual (ground truth) values.
    :param prices: A list or 1D NumPy array of prices corresponding to the targets.
    :return: The price-weighted MAE as a float.
    """
    # Ensure inputs are NumPy arrays
    predictions = np.array(predictions, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)
    prices = np.array(prices, dtype=np.float32)

    # Compute absolute error
    error = np.abs(targets - predictions)

    # Compute price-weighted error
    weighted_error = error * prices

    # Compute and return the mean of the weighted error
    return np.mean(weighted_error)

def local_iqr_clip(series, window=30, q1=0.25, q3=0.75, m=2.5):
    roll_q1 = series.rolling(window, center=True).quantile(q1)
    roll_q3 = series.rolling(window, center=True).quantile(q3)
    iqr = roll_q3 - roll_q1
    upper = roll_q3 + m * iqr
    return series.clip(0, upper)

class MultiTaskLossModule(nn.Module):
    def __init__(self):
        super(MultiTaskLossModule, self).__init__()
        self.alpha = 1.0
        self.beta = 1.0
        self.gamma = 5.0

    def forward(self, y_pred, y_true):
        # 내부에 바로 구현
        reg_pred   = y_pred[..., :2]
        reg_true   = y_true[..., :2]
        cls_pred   = y_pred[..., 2]
        cls_true   = y_true[..., 2]

        loss_demand   = F.mse_loss(reg_pred[..., 0], reg_true[..., 0])
        loss_discount = F.mse_loss(reg_pred[..., 1], reg_true[..., 1])
        loss_cls      = F.binary_cross_entropy_with_logits(cls_pred, cls_true)

        return self.alpha*loss_demand + self.beta*loss_discount + self.gamma*loss_cls


In [ ]:
TEST_DATE = pd.Timestamp('2023-01-01')
VAL_DATE_OUT = pd.Timestamp('2022-01-01')
VAL_DATE_IN = pd.Timestamp('2022-01-01')

dataset = pd.read_csv("data/all_master.csv", parse_dates=["date"])
dataset = dataset.sort_values(by=["city", "sku", "date"])

In [5]:
LABELS = {
    "JPN": [("2019-01-01", "2019-02-28")],
    "KOR": [("2018-01-15", "2018-04-15")],
    "USA": [("2020-01-01", "2020-04-30"),
            ("2021-03-01", "2021-05-31")]
}

dataset['is_event'] = 0

for country, periods in LABELS.items():
    for start, end in periods:
        start_dt = pd.to_datetime(start)
        end_dt   = pd.to_datetime(end)
        mask = (
            (dataset["country"] == country) &
            (dataset["date"] >= start_dt) &
            (dataset["date"] <= end_dt)   # end 포함
        )
        dataset.loc[mask, "is_event"] = 1

In [7]:
dataset.to_csv("data/train_demand.csv", index=False)

In [5]:
preprocess = 'none'
# preprocess = 'iqr'
dataset = pd.get_dummies(dataset, columns=['season'], prefix='season')
dataset = dataset[dataset['days_since_launch'] > 0]

dataset['month'] = dataset['date'].dt.month
dataset['day']   = dataset['date'].dt.day 
dataset['year']  = dataset['date'].dt.year
dataset['day_of_week'] = dataset['date'].dt.dayofweek

target_col = ['demand']
time_col = 'date'
group_cols = ['sku','city']

drop_cols = [
    # static
    # 'country', 
    'category', 'family', 'storage_gb', 'colour', 
    # numeric
    # 'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter', 'is_holiday',
    'avg_temp', 'humidity', 'precip_mm',
    'rain_mm', 'snow_mm', 'snow_depth_cm', 'pressure_msl', 'cloud_cover',
    'wind_speed_avg', 'wind_speed_max', 'wind_gust_max', 'wind_dir_mode',
    'shortwave_rad_MJ', 'vpd', 'cdd18', 'delta_temp',
    'delta_humidity',
]
drop_cols = []
# past_cols = ['EMA_30', 'MA_30']
past_cols = ['discount_pct', 'is_event']
future_cols = ['season_Fall', 'season_Spring', 'season_Summer', 'season_Winter', 'is_holiday','avg_temp', 'min_temp', 'max_temp', 'dewpoint', 'humidity', 'precip_mm',
       'rain_mm', 'snow_mm', 'snow_depth_cm', 'pressure_msl', 'cloud_cover',
       'wind_speed_avg', 'wind_speed_max', 'wind_gust_max', 'wind_dir_mode',
       'shortwave_rad_MJ', 'vpd', 'hdd18', 'cdd18', 'delta_temp',
       'delta_humidity', 'brent_usd', 'local_fx', 'spend_usd', 'days_since_launch',
       'month', 'day', "year", "day_of_week"
]

static_cols = ['country', 'category', 'family', 'storage_gb', 'colour', 'unit_price', 'life_days']

dataset = dataset.drop(columns=drop_cols)
future_cols = [col for col in future_cols if col not in drop_cols]
static_cols = [col for col in static_cols if col not in drop_cols]


if preprocess == 'clip':
    print('clip')
    low, high = dataset['demand'].quantile([0.00,0.95])
    dataset['demand'] = dataset['demand'].clip(low, high)
elif preprocess == 'iqr':
    print('iqr')
    dataset['demand'] = local_iqr_clip(dataset['demand'])
dataset['demand'] = np.log1p(dataset['demand'])
dataset['discount_pct'] = dataset['discount_pct']/100
    
series_raw, series, past_cov, future_cov = to_darts_time_series_group(
    dataset=dataset,
    target=target_col,
    time_col=time_col,
    group_cols=group_cols,
    past_cols=past_cols,
    future_cols=future_cols,
    static_cols=static_cols,
    freq='D', # daily
    encode_static_cov=True, # so that the models can use the categorical variables (Agency & Product)
)

In [ ]:
train_val, test = split_grouped_darts_time_series(
    series=series,
    split_date=TEST_DATE
)

train, val = split_grouped_darts_time_series(
    series=train_val,
    split_date=VAL_DATE_OUT
)


In [ ]:
early_stopping_args = {
    "monitor": "val_loss",
    "patience": 10,
    "min_delta": 1e-3,
    "mode": "min",
}

pl_trainer_kwargs = {
    "max_epochs": 100,
    "accelerator": "gpu", 
    "callbacks": [EarlyStopping(**early_stopping_args)],
    "enable_progress_bar":True
}

common_model_args = {
    "output_chunk_length": 7,
    "input_chunk_length": 84,
    # "output_chunk_length": 1,
    # "input_chunk_length": 90,
    "pl_trainer_kwargs": pl_trainer_kwargs,
    "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
    "force_reset": True,
    "batch_size": 512,
    "random_state": 42,
}

encoders = {
    "position": {"past": ["relative"], "future": ["relative"]},
    "transformer": Scaler(),
}

best_hp = {
 'optimizer_kwargs': {'lr':0.0001},
 'loss_fn': MAELoss(),
 'use_layer_norm': True,
 'use_reversible_instance_norm': True,
 'add_encoders':encoders,
}

In [8]:
past_cov = None if not past_cov else past_cov

start = time.time()

## COMMENT TO LOAD PRE-TRAINED MODEL
fit_mixed_covariates_model(
    model_cls = TiDEModel,
    common_model_args = common_model_args,
    specific_model_args = best_hp,
    model_name = 'TiDE_demand_onty_model',
    past_cov = past_cov,
    future_cov = future_cov,
    train_series = train,
    # train_series=train_val,
    val_series = val,
    # val_series=None,
)
time_tide = time.time() - start

number of `past_covariates` features is <= `temporal_width_past`, leading to feature expansion.number of covariates: 3, `temporal_width_past=4`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX PRO 6000 Blackwell Workstation Edition') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type             | Params | Mode 
--------------------------------------------------------------------
0  | criterion             | MAELoss          | 0      | train
1  | train_criterion       | MAELoss          | 0      | train
2  | val_criterion         | MAELoss          | 

Epoch 9:  10%|▉         | 261/2678 [00:12<01:54, 21.15it/s, train_loss=0.397, val_loss=0.367] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
best_tide = TiDEModel.load_from_checkpoint(model_name='TiDE_demand_onty_model', best=True)
preds_tide = best_tide.predict(
                    series            = train_val,
                    past_covariates   = past_cov,
                    future_covariates = future_cov,
                    n                 = test[0].n_timesteps 
                )

In [ ]:
from tqdm import tqdm

group_cols = ["sku","city"]  # 예시
groups_df = (
    dataset
    .loc[:, group_cols]
    .drop_duplicates()
    .sort_values(by=group_cols)   # from_group_dataframe 도 내부적으로 정렬하므로
    .reset_index(drop=True)
)

result = []
for i in tqdm(range(len(groups_df))):
    group_id = groups_df.iloc[i]
    
    pred = preds_tide[i].to_dataframe()
    pred = pred.reset_index()
    pred = pred.rename(columns={"index": "date"})
    # pred = pred.drop(columns=['discount_pct'])
    
    for j in range(len(pred)):
        result.append({
            "sku":  group_id["sku"],
            "city": group_id["city"],
            "date": pred['date'][j],
            "mean": pred['demand'][j],
            "discount_pct": pred['discount_pct'][j],
            "is_event": pred['is_event'][j],
        })

result_df = pd.DataFrame(result)
result_df['mean'] = np.expm1(result_df['mean'])
result_df['mean'] = result_df['mean'].round().astype(int)
result_df['date'] = pd.to_datetime(result_df['date'])
sub = pd.read_csv("extracted_contents/data/forecast_submission_template.csv", parse_dates=["date"])
sub.drop(columns=['mean'], inplace=True)
sub = sub.merge(result_df, on=['sku', 'city', 'date'], how='left')
sub.to_csv("result.csv", index=False)
sub.drop(columns=['discount_pct', 'is_event'], inplace=True)
sub.to_csv("forecast_submission_template.csv", index=False)

# TFT

In [ ]:
# PyTorch Lightning Trainer arguments
early_stopping_args = {
    "monitor": "val_loss",
    "patience": 50,
    "min_delta": 1e-3,
    "mode": "min",
}

pl_trainer_kwargs = {
    "max_epochs": 200,
    "accelerator": "gpu", # uncomment for gpu use
    "callbacks": [EarlyStopping(**early_stopping_args)],
    "enable_progress_bar":True
}

common_model_args = {
    "output_chunk_length": 1,
    "input_chunk_length": 30,
    "pl_trainer_kwargs": pl_trainer_kwargs,
    "save_checkpoints": True,  # checkpoint to retrieve the best performing model state,
    "force_reset": True,
    "batch_size": 128,
    "random_state": 42,
}

In [ ]:
encoders = {
    "position": {"past": ["relative"], "future": ["relative"]},
    "transformer": Scaler(),
}

best_hp = {
 'optimizer_kwargs': {'lr':0.0001},
 'loss_fn': MultiTaskLossModule(),
 'use_reversible_instance_norm': True,
 'add_encoders':encoders,
 }

In [ ]:
past_cov = None if not past_cov else past_cov

start = time.time()
## COMMENT TO LOAD PRE-TRAINED MODEL
fit_mixed_covariates_model(
    model_cls = TFTModel,
    common_model_args = common_model_args,
    specific_model_args = best_hp,
    model_name = 'TFT_model',
    past_cov = past_cov,
    future_cov = future_cov,
    train_series = train,
    val_series = val,
)
time_tft = time.time() - start

In [ ]:
best_tft = TFTModel.load_from_checkpoint(model_name='TFT_model', best=True)
preds_tft = best_tft.predict(
                    series            = train_val,
                    past_covariates   = past_cov,
                    future_covariates = future_cov,
                    n                 = test[0].n_timesteps
                )

In [ ]:
from tqdm import tqdm

group_cols = ["sku","city"]  # 예시
groups_df = (
    dataset
    .loc[:, group_cols]
    .drop_duplicates()
    .sort_values(by=group_cols)   # from_group_dataframe 도 내부적으로 정렬하므로
    .reset_index(drop=True)
)

result = []
for i in tqdm(range(len(groups_df))):
    group_id = groups_df.iloc[i]
    
    pred = preds_tft[i].to_dataframe()
    pred = pred.reset_index()
    pred = pred.rename(columns={"index": "date"})
    pred = pred.drop(columns=['discount_pct'])
    # pred['demand'] = np.exp(pred['demand'])
    
    for j in range(len(pred)):
        result.append({
            "sku":  group_id["sku"],
            "city": group_id["city"],
            "date": pred['date'][j],
            "mean": pred['demand'][j],
        })

result_df = pd.DataFrame(result)
result_df['mean'] = np.expm1(result_df['mean'])
result_df['mean'] = result_df['mean'].round().astype(int)
result_df['date'] = pd.to_datetime(result_df['date'])
sub = pd.read_csv("extracted_contents/data/forecast_submission_template.csv", parse_dates=["date"])
sub.drop(columns=['mean'], inplace=True)
sub = sub.merge(result_df, on=['sku', 'city', 'date'], how='left')
sub.to_csv("forecast_submission_template.csv", index=False)